In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
import re
import os
import itertools
from tqdm.auto import tqdm
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.io as pio
from typing import Any, Dict, List, Union

In [3]:
publishers = ["HarperCollins", "Little, Brown and Company", "Tor Books", "Doubleday", "Bloomsbury Publishing"]  # Disney Hyperion Books (Disney Publishing Group)、Oxford World's Classics 沒有書在榜
formats = ["Paperback", "Hardcover", "Kindle"]
features = ["category_rank", "whole_rank", "original_price", "publish_year"]

In [4]:
data_2dep_ori = pd.read_csv("Amazon_dep2_all.csv", encoding="latin-1", error_bad_lines=False)

C:\Users\USER\AppData\Local\Temp\ipykernel_4836\2326627645.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data_2dep_ori = pd.read_csv("Amazon_dep2_all.csv", encoding="latin-1", error_bad_lines=False)
Skipping line 5485: expected 9 fields, saw 13
Skipping line 10963: expected 9 fields, saw 11
Skipping line 10964: expected 9 fields, saw 11
Skipping line 10965: expected 9 fields, saw 11
Skipping line 10966: expected 9 fields, saw 11
Skipping line 10967: expected 9 fields, saw 11
Skipping line 10968: expected 9 fields, saw 11
Skipping line 25045: expected 9 fields, saw 10
Skipping line 46904: expected 9 fields, saw 10
Skipping line 46905: expected 9 fields, saw 10
Skipping line 46906: expected 9 fields, saw 10
Skipping line 49409: expected 9 fields, saw 11
Skipping line 59516: expected 9 fields, saw 10

Skipping line 74577: expected 9 fields, saw 10
Skipping line 74578: expected 9 fields, sa

In [5]:
def formatting_data(data: pd.DataFrame) -> pd.DataFrame:
    data = data.rename(columns={"all_rank": "whole_rank", "before_discount": "original_price", "new_price": "selling_price", "department": "category"})

    def clean_price_column(price_series: pd.Series) -> pd.Series:
        price_series_cleaned = price_series.str.extract(r"(\$?[0-9]+(?:\.[0-9]{1,2})?)", expand=False)
        price_series_cleaned = price_series_cleaned.str.replace("$", "", regex=False).astype(float)
        price_series_cleaned = price_series_cleaned.where((price_series_cleaned >= 0) & (price_series_cleaned < 200), np.nan)
        return price_series_cleaned

    data["whole_rank"] = data["whole_rank"].apply(lambda x: int("".join(filter(str.isdigit, str(x)))) if "#" in str(x) else None).astype(float)
    data["whole_rank"] = data["whole_rank"].where(data["whole_rank"] > 0, np.nan)
    data["format"] = data["format"].apply(lambda x: next((f for f in formats if f in str(x)), "Other Format")).astype(str)
    data["original_price"] = clean_price_column(data["original_price"])
    data["selling_price"] = clean_price_column(data["selling_price"])
    data["original_price"] = data["original_price"].fillna(data["selling_price"])
    data["publisher"] = data["other_list"].apply(lambda x: next((p for p in publishers if f"Publisher:{p}".lower() in str(x).lower()), "Other Publishers")).astype(str)
    data["publish_year"] = data["other_list"].str.extract(r"\b(\d{4})\b", expand=False).astype(float)
    data["publish_year"] = data["publish_year"].where((data["publish_year"] >= 1600) & (data["publish_year"] <= 2025), np.nan)
    data["discount_rate"] = data["selling_price"] / data["original_price"]
    data["discount_rate"] = data["discount_rate"].where((data["discount_rate"] >= 0) & (data["discount_rate"] <= 1), np.nan).astype(float)

    return data.drop_duplicates()


data_2dep = formatting_data(data_2dep_ori)
data_2dep.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109447 entries, 0 to 109447
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   whole_rank      104914 non-null  float64
 1   dep_rank        109447 non-null  object 
 2   format          109447 non-null  object 
 3   category        109447 non-null  object 
 4   title           109447 non-null  object 
 5   original_price  108129 non-null  float64
 6   selling_price   108126 non-null  float64
 7   scrape_time     109444 non-null  object 
 8   other_list      109444 non-null  object 
 9   publisher       109447 non-null  object 
 10  publish_year    95487 non-null   float64
 11  discount_rate   94707 non-null   float64
dtypes: float64(5), object(7)
memory usage: 10.9+ MB


In [6]:
data_2dep_biz = data_2dep[data_2dep["category"].str.contains("Business & Money")]

In [48]:
len(data_2dep_biz["title"].unique())

1364

In [45]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 給定的變量和數據框架
publishers = ["HarperCollins", "Little, Brown and Company", "Tor Books", "Doubleday", "Bloomsbury Publishing"]
formats = ["Paperback", "Hardcover", "Kindle"]
features = ["category_rank", "whole_rank", "original_price", "publish_year"]
bins_ranges = {
    "whole_rank": [1, 100, 1000, 10000, 100000, 1000000, 10000000, 100000000],
    "original_price": [0, 5, 10, 15, 20, 25, 30, 100],
    "selling_price": [0, 5, 10, 20, 30, 40, 100],
    "publish_year": [0, 1990, 1995, 2000, 2005, 2010, 2015, 2020, 2025],
}


# 新的plot_pattern函數
def plot_pattern(data: pd.DataFrame, data_name: str):
    # 定義4行3列，每列表示一個格式，每行表示不同的特性
    fig = make_subplots(rows=4, cols=3, subplot_titles=[f"{format} - {feature}" for feature in ["whole_rank", "original_price", "selling_price", "publish_year"] for format in formats])

    col = 1
    for format in formats:
        row = 1
        data_format = data[data["format"] == format]
        for feature in ["whole_rank", "original_price", "selling_price", "publish_year"]:
            # 計算IQR
            Q1 = data_format[feature].quantile(0.25)
            Q3 = data_format[feature].quantile(0.75)
            IQR = Q3 - Q1

            # 去除極端值
            filtered_data = data_format[(data_format[feature] >= Q1 - 1.5 * IQR) & (data_format[feature] <= Q3 + 1.5 * IQR)].dropna(subset=[feature, "discount_rate"])

            bins = pd.cut(filtered_data[feature], bins=bins_ranges[feature])
            for bin in bins.unique().sort_values():
                bin_data = filtered_data[bins == bin]["discount_rate"]
                bin_count = len(bin_data)
                bin_percentage = bin_count / len(filtered_data) * 100
                fig.add_trace(go.Box(y=bin_data, name=f"{str(bin)} ({bin_count}; {bin_percentage:.2f}%)", marker_color="blue"), row=row, col=col)
            row += 1
        col += 1

    # 調整整體布局
    fig.update_yaxes(range=[-0.1, 1.1])
    fig.update_layout(height=1200, width=1200, title_text=f"{data_name} - Discount Rate Distribution", showlegend=False)
    fig.write_image("0624_figs/discount_rate distribution.svg")
    fig.show()


plot_pattern(data_2dep_biz, "[2dep Business & Money]")

In [8]:
def consolidate_books(data: pd.DataFrame) -> pd.DataFrame:
    # 初始化一個空列表，用來存放處理後的書籍數據，每個元素是一個字典，對應一本書的整合數據。
    consolidated_list: List[Dict[str, Any]] = []

    # 對原始數據按照書名(title)進行分組，每一組代表同一本書的不同條目。
    for title, group in data.groupby("title"):
        # 創建一個字典來存放當前書的整合數據。
        # key為書名，value為書名本身。
        # category欄位結合同一書名下所有不同類別，使用分號隔開。
        book_data: Dict[str, Union[str, float]] = {"title": title, "category": ";".join(group["category"].astype(str).unique())}

        # 再次對同一書名的數據按照版本(format)進行分組。
        for format_group in group.groupby("format"):
            # 格式化版本名稱，將空格替換為下劃線並轉為小寫，以便後續作為欄位名使用。
            format_type: str = format_group[0].replace(" ", "_").lower()
            # 獲取具體某一版本的所有條目。
            format_data: pd.DataFrame = format_group[1]
            # 從這些條目中選擇第一條作為代表。
            representative: pd.Series = format_data.iloc[0]

            # 遍歷需要整合的欄位名稱列表。
            for field in ["original_price", "selling_price", "other_list", "publisher", "publish_year", "discount_rate", "whole_rank"]:
                # 為每個欄位名稱加上版本前綴，並從代表條目中提取對應的值，存放到book_data字典中。
                book_data[f"{format_type}-{field}"] = representative[field]

        # 將處理後的書籍數據加入到列表中。
        consolidated_list.append(book_data)

    # 將列表轉換為DataFrame並返回。
    return pd.DataFrame(consolidated_list)


data_2dep_biz_con = consolidate_books(data_2dep_biz)

In [10]:
data_2dep_biz_con["hard_over_paper"] = data_2dep_biz_con["hardcover-selling_price"] / data_2dep_biz_con["paperback-selling_price"]

In [21]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def plot_scatter_distributions(data: pd.DataFrame) -> None:
    # Define plot variables and book types
    variables = ["whole_rank", "original_price", "selling_price", "publish_year", "discount_rate"]
    book_types = ["paperback", "hardcover"]

    # Create a subplot with 4 rows and 2 columns
    fig = make_subplots(rows=len(variables), cols=2, subplot_titles=[f"{var} (Paperback)" for var in variables] + [f"{var} (Hardcover)" for var in variables])

    for row, var in enumerate(variables, start=1):
        for col, book_type in enumerate(book_types, start=1):
            var_name = f"{book_type}-{var}"

            # Remove top 5% extreme values
            threshold = data[var_name].quantile(0.95)
            filtered_data = data[data[var_name] <= threshold]

            # Define color based on column (1 for left, 2 for right)
            marker_color = "blue" if col == 1 else "red"

            scatter = go.Scatter(
                x=filtered_data["hard_over_paper"],
                y=filtered_data[var_name],
                mode="markers",
                marker=dict(color=marker_color),
                showlegend=False,
                name=var_name,
            )

            fig.add_trace(scatter, row=row, col=col)

        # Get y-axis range after removing extremes and add buffer
        y_min = min(data[f"paperback-{var}"].quantile(0.05), data[f"hardcover-{var}"].quantile(0.05))
        y_max = max(data[f"paperback-{var}"].quantile(0.95), data[f"hardcover-{var}"].quantile(0.95))
        buffer = (y_max - y_min) * 0.1  # 10% buffer

        fig.update_yaxes(range=[y_min - buffer, y_max + buffer], row=row, col=1)
        fig.update_yaxes(range=[y_min - buffer, y_max + buffer], row=row, col=2)

    fig.update_xaxes(title_text="Hardcover over Paperback", row=len(variables), col=1)
    fig.update_xaxes(title_text="Hardcover over Paperback", row=len(variables), col=2)

    fig.update_layout(height=4000, width=1600, title_text="Distributions of Variables")
    fig.write_image(f"0705_figs/all scatter.svg")
    fig.show()


# Assuming data_2dep_fic is your DataFrame
plot_scatter_distributions(data_2dep_biz_con)

In [14]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def plot_scatter_distributions(data: pd.DataFrame) -> None:
    # Define plot variables and book types
    variables = ["whole_rank", "original_price", "selling_price", "publish_year", "discount_rate"]
    book_types = ["paperback", "hardcover"]

    for var in variables:
        fig = make_subplots(rows=1, cols=2, subplot_titles=("Paperback", "Hardcover"))

        for i, book_type in enumerate(book_types):
            var_name = f"{book_type}-{var}"

            # Remove top 5% extreme values
            threshold = data[var_name].quantile(0.95)
            filtered_data = data[data[var_name] <= threshold]

            scatter = go.Scatter(
                x=filtered_data["hard_over_paper"],
                y=filtered_data[var_name],
                mode="markers",
                showlegend=False,
                name=var_name,
            )

            fig.add_trace(scatter, row=1, col=i + 1)

        # Get y-axis range after removing extremes
        y_min = min(data[f"paperback-{var}"].quantile(0.05), data[f"hardcover-{var}"].quantile(0.05))
        y_max = max(data[f"paperback-{var}"].quantile(0.95), data[f"hardcover-{var}"].quantile(0.95))

        fig.update_xaxes(title_text="Hardcover over Paperback", row=1, col=1)
        fig.update_xaxes(title_text="Hardcover over Paperback", row=1, col=2)
        fig.update_yaxes(title_text=var, range=[y_min, y_max], row=1, col=1)
        fig.update_yaxes(title_text=var, range=[y_min, y_max], row=1, col=2)

        fig.update_layout(title_text=f"{var} Distributions", height=800, width=1600)
        fig.write_image(f"0705_figs/{var} scatter.svg")
        fig.show()


# Assuming data_2dep_fic is your DataFrame
plot_scatter_distributions(data_2dep_biz_con)

In [25]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


def plot_rank_price_ratio(data_con: pd.DataFrame) -> None:
    # 提取相關欄位
    paperback_rank = data_con["paperback-whole_rank"]
    hardcover_rank = data_con["hardcover-whole_rank"]
    paperback_price = data_con["paperback-selling_price"]
    hardcover_price = data_con["hardcover-selling_price"]
    title = data_con["title"]

    # 計算價格比例
    price_ratio = hardcover_price / paperback_price

    # 剔除缺失值
    valid = ~(paperback_rank.isna() | hardcover_rank.isna() | price_ratio.isna())
    paperback_rank = paperback_rank[valid]
    hardcover_rank = hardcover_rank[valid]
    price_ratio = price_ratio[valid]
    title = title[valid]

    # 剔除兩軸超過 1M 的值
    valid = (paperback_rank <= 1_000_000) & (hardcover_rank <= 1_000_000)
    paperback_rank = paperback_rank[valid]
    hardcover_rank = hardcover_rank[valid]
    price_ratio = price_ratio[valid]
    title = title[valid]

    # 計算價格比例的百分位數
    price_ratio_percentile = price_ratio.rank(pct=True)

    # 自定義色標刻度標籤
    tickvals = [i / 10 for i in range(11)]
    ticktext = [f"{price_ratio.quantile(p):.2f}" for p in tickvals]

    # 建立 Plotly 圖表
    fig = px.scatter(
        x=paperback_rank,
        y=hardcover_rank,
        color=price_ratio_percentile,
        color_continuous_scale="RdYlBu_r",
        title="Book Rank Comparison",
        labels={"x": "Paperback Whole Rank", "y": "Hardcover Whole Rank", "color": "Price Ratio"},
        hover_data={"Price Ratio": price_ratio, "Title": title},
    )

    # 自定義色標刻度
    fig.update_coloraxes(colorbar=dict(tickvals=tickvals, ticktext=ticktext))

    # 增加 x=y 的直線
    fig.add_trace(
        go.Scatter(x=[min(paperback_rank), max(paperback_rank)], y=[min(paperback_rank), max(paperback_rank)], mode="lines", line=go.scatter.Line(color="black", dash="dash"), showlegend=False)
    )

    # 確保兩軸刻度單位相同，且繪圖區域為正方形
    fig.update_layout(height=800, width=800, xaxis=dict(scaleanchor="y", scaleratio=1), yaxis=dict(scaleanchor="x", scaleratio=1))

    # 顯示圖表
    fig.show()

    # 儲存圖表
    fig.write_image("0705_figs/scatter_rank.svg")
    fig.write_html("0705_figs/scatter_rank.html")


# 示例使用 data_2dep_biz_con 資料框
plot_rank_price_ratio(data_2dep_biz_con)